## Setup Github

Import the github repository and move it into the local directory (also remove the defult samples directoy if it exists)

In [29]:
from getpass import getpass
import os

if not os.path.isdir(".git"):
    user = "Boyne272"
    password = getpass('github password')
    os.environ['GITHUB_AUTH'] = user + ':' + password

    # clone the repo
    !git clone https://$GITHUB_AUTH@github.com/msc-acse/acse-8-miniproject-softmax.git repo

    # move the repo up one
    !mv repo/* .
    !mv repo/.git .

    # delete un needed stuff
    !rm -r repo
    !rm -r sample_data/
!git checkout richard
!git pull origin richard
!git log -2

D	.gitignore
Already on 'richard'
Your branch is up to date with 'origin/richard'.
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/msc-acse/acse-8-miniproject-softmax
 * branch            richard    -> FETCH_HEAD
   fdb9787..92fbe66  richard    -> origin/richard
Updating fdb9787..92fbe66
Fast-forward
 models/combined_predictions.csv | 20002 +++++++++++++++++++-------------------
 1 file changed, 10001 insertions(+), 10001 deletions(-)
commit 92fbe666b8354237db02310e739edc644e88a49f (HEAD -> richard, origin/richard)
Author: Boyne272 <boynerichard@yahoo.co.uk>
Date:   Thu May 23 21:09:12 2019 +0100

    minor correction

commit fdb9787234365c08c61b2ddfa10bd01ad7fd773e
Author: Boyne272 <boynerichard@yahoo.co.uk>
Date:   Thu May 23 21:06:38 2019 +0100

    added final models to comparison


## Setup Imports

Ipython imports and functions

In [11]:
# ipython setup
!pip install pycm livelossplot
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random', 'Normalize']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Module imports

In [0]:
# imports
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from torchvision.transforms import Compose, ToTensor, Normalize, RandomRotation, RandomCrop, ToPILImage, Pad

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# imports from github
from tools import *

Check runtime device

In [14]:
# check running device
device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

Cuda installed! Running on GPU!


Mount google drive if wanted

In [13]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


## Load Data

In [0]:
train_feat = np.load("/content/data/kmnist-train-imgs.npy")
train_targ = np.load("/content/data/kmnist-train-labels.npy")
test_feat = np.load("/content/data/kmnist-test-imgs.npy")

In [0]:
# # check what the data looks like
# for data, data_name in zip([train_targ, test_feat, train_feat], 
#                            ["train_targ", "test_feat", "train_feat"]):
    
#     print(data_name)
    
#     for name, code in zip(["shape", "mean", "std", "max", "min", "type", "cuda"],
#                           ["data.shape", "data.mean()", "data.std()", 
#                            "data.max()", "data.min()", "data.dtype"]):
#         try:
#             exec("print('\t', name, '\t:\t', " + code + ")")
#         except:
#             print("\t", name, "\t:\t undefined")

## Define the Models

## Load the models

## Load the CSV files

In [0]:
import pandas as pd

db = pd.read_csv("/content/models/combined_predictions.csv")

In [34]:
db.corr()

,DAB Drop Aug (98.88%),DAB Drop Aug full (98.2%),DAB opt SGD 2,DAB opt ADAM 1,DAB opt ADAM 2,AlexNet 7 Adam,AlexNet 7 SGD,AlexNet 7 Adam full,AlexNet 7 SGD full,Super Awesome Dodgie thing no pad,Super Awesome Dodgie thing pad,AlexNet 6 wide
DAB Drop Aug (98.88%),1.000000,0.988160,0.987419,0.986022,0.986022,0.987594,0.988542,0.985701,0.984890,0.987295,0.984803,0.984671
DAB Drop Aug full (98.2%),0.988160,1.000000,0.985661,0.983934,0.983934,0.985670,0.986916,0.981631,0.984927,0.985840,0.982116,0.980758
DAB opt SGD 2,0.987419,0.985661,1.000000,0.983237,0.983237,0.984192,0.985791,0.984815,0.982747,0.987268,0.981633,0.979733
DAB opt ADAM 1,0.986022,0.983934,0.983237,1.000000,1.000000,0.984980,0.985189,0.982190,0.984366,0.984201,0.982942,0.980670
DAB opt ADAM 2,0.986022,0.983934,0.983237,1.000000,1.000000,0.984980,0.985189,0.982190,0.984366,0.984201,0.982942,0.980670
AlexNet 7 Adam,0.987594,0.985670,0.984192,0.984980,0.984980,1.000000,0.987954,0.986296,0.985068,0.985386,0.980343,0.983232
AlexNet 7 SGD,0.988542,0.986916,0.985791,0.985189,0.985189,0.987954,1.000000,0.987855,0.987465,0.985860,0.981396,0.984165
AlexNet 7 Adam full,0.985701,0.981631,0.984815,0.982190,0.982190,0.986296,0.987855,1.000000,0.983851,0.984448,0.978563,0.982342
AlexNet 7 SGD full,0.984890,0.984927,0.982747,0.984366,0.984366,0.985068,0.987465,0.983851,1.000000,0.984754,0.979847,0.981868
Super Awesome Dodgie thing no pad,0.987295,0.985840,0.987268,0.984201,0.984201,0.985386,0.985860,0.984448,0.984754,1.000000,0.984643,0.979731


In [36]:
names = db.keys()
names

Index(['DAB Drop Aug (98.88%)', 'DAB Drop Aug full (98.2%)', 'DAB opt SGD 2',
       'DAB opt ADAM 1', 'DAB opt ADAM 2', 'AlexNet 7 Adam', 'AlexNet 7 SGD',
       'AlexNet 7 Adam full', 'AlexNet 7 SGD full',
       'Super Awesome Dodgie thing no pad', 'Super Awesome Dodgie thing pad',
       'AlexNet 6 wide'],
      dtype='object')

In [0]:
ensemble_names = ["DAB opt SGD 2", "DAB opt ADAM 1", "AlexNet 7 Adam", "AlexNet 7 SGD"]

In [39]:
ensemble = db[ensemble_names]
ensemble

,DAB opt SGD 2,DAB opt ADAM 1,AlexNet 7 Adam,AlexNet 7 SGD
0,2,2,2,2
1,9,9,9,9
2,3,3,3,3
3,8,8,8,8
4,3,3,3,3
5,3,3,3,3
6,8,8,8,8
7,3,3,3,3
8,2,2,2,2
9,5,5,5,5


In [61]:
output = []
for index, row in ensemble.iterrows():
#     np.unique(row)
    uniq, cnt = np.unique(row.values, return_counts=True)
    if len(uniq) == 1:
        output.append(*uniq)
    
    elif cnt[0]==2:
        output.append(np.random.choice(uniq))
#         pass
    else:
#         print(row.values)
        output.append(uniq[np.argmax(cnt)])
#     print(np.unique(row))
print(output)

[2, 9, 3, 8, 3, 3, 8, 3, 2, 5, 6, 3, 3, 3, 1, 5, 4, 8, 6, 3, 7, 5, 7, 5, 7, 0, 3, 5, 4, 4, 6, 8, 5, 9, 5, 6, 6, 3, 0, 6, 4, 7, 8, 1, 1, 6, 9, 0, 5, 0, 1, 1, 4, 8, 4, 1, 3, 3, 3, 4, 6, 4, 8, 3, 6, 5, 3, 4, 6, 1, 5, 9, 9, 7, 3, 2, 3, 7, 6, 3, 4, 1, 1, 5, 9, 6, 5, 1, 0, 1, 2, 7, 1, 6, 7, 3, 6, 5, 9, 2, 0, 8, 8, 2, 3, 5, 9, 1, 0, 8, 5, 2, 0, 0, 5, 5, 2, 9, 0, 9, 3, 6, 9, 5, 6, 2, 0, 7, 6, 2, 9, 0, 4, 1, 3, 4, 6, 9, 9, 5, 8, 4, 5, 9, 6, 8, 2, 4, 7, 3, 5, 1, 8, 5, 8, 6, 3, 2, 7, 6, 2, 5, 6, 7, 5, 2, 8, 0, 6, 6, 4, 1, 9, 5, 8, 6, 5, 6, 8, 7, 4, 4, 0, 6, 5, 7, 2, 6, 2, 1, 8, 9, 0, 7, 4, 3, 0, 4, 6, 3, 1, 3, 5, 0, 8, 8, 5, 8, 2, 2, 3, 2, 7, 5, 7, 5, 4, 5, 4, 7, 2, 3, 7, 5, 8, 9, 0, 7, 5, 0, 1, 6, 6, 3, 0, 9, 8, 4, 9, 0, 2, 0, 3, 0, 0, 6, 8, 3, 7, 9, 4, 4, 0, 5, 5, 9, 6, 4, 0, 4, 2, 7, 3, 2, 6, 7, 0, 9, 1, 5, 4, 5, 5, 9, 8, 7, 9, 0, 3, 3, 6, 1, 6, 8, 7, 7, 9, 2, 6, 5, 2, 7, 5, 7, 3, 8, 0, 8, 3, 0, 3, 2, 8, 7, 9, 2, 0, 2, 1, 0, 0, 7, 8, 7, 9, 4, 2, 2, 9, 8, 6, 6, 8, 4, 9, 7, 4, 3, 5, 2, 2, 8, 2, 

In [64]:
db["ensemble"] = output
db.corr()

,DAB Drop Aug (98.88%),DAB Drop Aug full (98.2%),DAB opt SGD 2,DAB opt ADAM 1,DAB opt ADAM 2,AlexNet 7 Adam,AlexNet 7 SGD,AlexNet 7 Adam full,AlexNet 7 SGD full,Super Awesome Dodgie thing no pad,Super Awesome Dodgie thing pad,AlexNet 6 wide,ensemble
DAB Drop Aug (98.88%),1.000000,0.988160,0.987419,0.986022,0.986022,0.987594,0.988542,0.985701,0.984890,0.987295,0.984803,0.984671,0.989995
DAB Drop Aug full (98.2%),0.988160,1.000000,0.985661,0.983934,0.983934,0.985670,0.986916,0.981631,0.984927,0.985840,0.982116,0.980758,0.988737
DAB opt SGD 2,0.987419,0.985661,1.000000,0.983237,0.983237,0.984192,0.985791,0.984815,0.982747,0.987268,0.981633,0.979733,0.989688
DAB opt ADAM 1,0.986022,0.983934,0.983237,1.000000,1.000000,0.984980,0.985189,0.982190,0.984366,0.984201,0.982942,0.980670,0.989897
DAB opt ADAM 2,0.986022,0.983934,0.983237,1.000000,1.000000,0.984980,0.985189,0.982190,0.984366,0.984201,0.982942,0.980670,0.989897
AlexNet 7 Adam,0.987594,0.985670,0.984192,0.984980,0.984980,1.000000,0.987954,0.986296,0.985068,0.985386,0.980343,0.983232,0.992785
AlexNet 7 SGD,0.988542,0.986916,0.985791,0.985189,0.985189,0.987954,1.000000,0.987855,0.987465,0.985860,0.981396,0.984165,0.992534
AlexNet 7 Adam full,0.985701,0.981631,0.984815,0.982190,0.982190,0.986296,0.987855,1.000000,0.983851,0.984448,0.978563,0.982342,0.987745
AlexNet 7 SGD full,0.984890,0.984927,0.982747,0.984366,0.984366,0.985068,0.987465,0.983851,1.000000,0.984754,0.979847,0.981868,0.988131
Super Awesome Dodgie thing no pad,0.987295,0.985840,0.987268,0.984201,0.984201,0.985386,0.985860,0.984448,0.984754,1.000000,0.984643,0.979731,0.989139


In [65]:
save_csv(output, "ensemble", "/content/gdrive/My Drive/models/")

successfully saved in /content/gdrive/My Drive/models/ensemble.csv


In [66]:
keys = ["AlexNet 7 Adam", "AlexNet 7 SGD"]
alex_7 = db[keys]


Index(['DAB Drop Aug (98.88%)', 'DAB Drop Aug full (98.2%)', 'DAB opt SGD 2',
       'DAB opt ADAM 1', 'DAB opt ADAM 2', 'AlexNet 7 Adam', 'AlexNet 7 SGD',
       'AlexNet 7 Adam full', 'AlexNet 7 SGD full',
       'Super Awesome Dodgie thing no pad', 'Super Awesome Dodgie thing pad',
       'AlexNet 6 wide'],
      dtype='object')